## Synthetic data

In [1]:
%load_ext autoreload 
%autoreload 2

import sys
import ast

import pandas as pd
import numpy as np

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
from src import aposteriori

import tasks.run_helper

In [2]:
syn_df = pd.read_csv(
    "../data/synthetic.csv",
    converters={
        "annot_personality_characteristics": ast.literal_eval,
        "Toxicity": ast.literal_eval,
        "Argument Quality": ast.literal_eval,
        "age_annot": ast.literal_eval,
        "sex_annot": ast.literal_eval,
        "sexual_orientation_annot": ast.literal_eval,
        "current_employment_annot": ast.literal_eval,
        "education_level_annot": ast.literal_eval,
    },
)
syn_df["comment_key"] = syn_df.message + syn_df.conv_id
syn_df["fake_index"] = 1

syn_df.Toxicity = syn_df.Toxicity.apply(lambda x: [int(tox) for tox in x])

syn_df.age_annot = syn_df.age_annot.apply(
    lambda ls: [int(x) for x in ls]
).apply(lambda x: pd.cut(x, bins=4))

syn_df["random"] = tasks.preprocessing.get_rand_col(df, "sex_annot")

In [3]:
sdb_columns = [
    "age_annot",
    "sexual_orientation_annot",
    "education_level_annot",
    "current_employment_annot",
    "sex_annot",
]
res = tasks.run_helper.run_all_results(
    df=syn_df,
    sdb_columns=sdb_columns,
    discussion_id_col="fake_index",
    value_col="Toxicity",
    comment_key_col="comment_key",
)
res

Evaluating SDB dimensions:   0%|          | 0/5 [00:00<?, ?it/s]

1          
                                                  kappa    pvalue
sdb_column                                                       
age_annot                (20.956, 32.0]        0.006701  0.654453
                         (32.0, 43.0]          0.044338  0.654453
                         (43.0, 54.0]          0.018339  0.654453
                         (54.0, 65.0]         -0.041638  0.654453
current_employment_annot Botanist              0.000000  1.000000
                         Cybersecurity Expert  0.000000  1.000000
                         Farmer                0.000000  1.000000
                         Game Developer        0.000000  1.000000
                         Historian             0.000000  1.000000
                         Poet                  0.000000  1.000000
                         Registered Nurse      0.000000  1.000000
                         Research Scientist    0.000000  1.000000
                         Retired Philosopher   0.000000  1.000000
                         Stock Trader          0.000000  1.000000
                         Travel Blogger        0.000000  1.000000
education_level_annot    Bachelor's            0.020260  0.820180
                         Master's             -0.012473  0.820180
                         No formal education   0.000000  1.000000
                         PhD                  -0.033979  0.820180
                         Some College          0.041981  0.820180
sex_annot                female                0.031191  0.994363
                         male                 -0.000440  0.994363
                         non-binary            0.000000  1.000000
sexual_orientation_annot Asexual               0.000000  1.000000
                         Bisexual              0.000000  1.000000
                         Heterosexual         -0.018541  1.000000
                         Homosexual            0.000000  1.000000
                         Lesbian               0.000000  1.000000
                         Pansexual             0.000000  1.000000

In [ ]:
tasks.run_helper.run_result(
    syn_df,
    discussion_id_col="fake_index",
    sdb_column="random",
    value_col="Toxicity",
    comment_key_col="comment_key",
)